In [1]:
import pandas as pd
import AtlasAnalysisFunctions as AAF
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# GEA on the deleted proteins per class

Idea: If the filtering procedure enhances classification, yet shuffled filtering does not, the protein deletions must be class specific. <br>
This will be evaluated here.

In [2]:
# Minimal preprocessing
base = pd.read_csv("base_ML_dataset.csv")
metadata = base.iloc[:, -2:]
metadata = metadata.reset_index().rename(columns = {"index": "assay_id"})
metadata.loc[metadata.tissue_type == "T-cell", "tissue_type"] = "blood"
labels = base.pop("tissue_type")
data = base.iloc[:, :-1]
data.fillna(0, inplace = True)
data.head()
full_df = pd.concat([data, labels], axis = 1)
full_df = full_df[~full_df.tissue_type.isin(["trophoblast","prostate", "fetal kidney", "lung", "fibroblast", "/"])]
full_df.loc[full_df.tissue_type == "T-cell", ["tissue_type"]] = "blood"
labels = full_df.pop("tissue_type")
data = full_df
metadata = metadata.loc[metadata.index.isin(data.index),:]

In [4]:
f50_global = AAF.ModelModule(data, metadata)
f50_global.set_labels("tissue_type")
f50_global.filter_dataset("reoccur global", percentage_reoccurence=.50)

Filtered dataset on reoccur global stored in 'filtered_dataset'


In [3]:
prot_go_mapping = pd.read_csv("uniprot_GO_mapping.csv")
prot_go_mapping.head()

,uniprot_id,GO_ID,Aspect
0,A0AVT1,GO:0005515,F
1,A0AVT1,GO:0005524,F
2,A0AVT1,GO:0019780,F
3,A0AVT1,GO:0006511,P
4,A0AVT1,GO:0007612,P


In [ ]:
F_mask = prot_go_mapping.Aspect == "F"
P_mask = prot_go_mapping.Aspect == "P"
C_mask = prot_go_mapping.Aspect == "C"

In [17]:
prot_go_mapping[prot_go_mapping.uniprot_id.isin(f50_global.deletions_per_class["liver"])]

,uniprot_id,GO_ID,Aspect
134,A6NHT5,GO:0000981,F
135,A6NHT5,GO:0007420,P
136,A6NHT5,GO:0007566,P
137,A6NHT5,GO:0030154,P
138,A6NHT5,GO:0042472,P
...,...,...,...
217125,Q9H069,GO:0005515,F
217126,Q9H069,GO:0005737,C
217127,Q9H069,GO:0005930,C
217128,Q9H069,GO:0036126,C


In [20]:
GO_del_df = pd.DataFrame(columns = ["uniprot_id", "GO_ID", "Aspect", "tissue"])
for tissue, del_prots in f50_global.deletions_per_class.items():
    del_df = prot_go_mapping[prot_go_mapping.uniprot_id.isin(del_prots)]
    del_df["tissue"] = tissue
    GO_del_df = pd.concat([GO_del_df, del_df])

/home/compomics/miniconda3/envs/ionbot/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [24]:
functional_GO = GO_del_df[GO_del_df.Aspect == "F"]
process_GO = GO_del_df[GO_del_df.Aspect == "P"]
compartment_GO = GO_del_df[GO_del_df.Aspect == "C"]

In [33]:
functional_GO.GO_ID.value_counts()

GO:0005515    41502
GO:0046872     7894
GO:0005524     5728
GO:0042802     4983
GO:0003723     4855
              ...  
GO:0008903        1
GO:0004408        1
GO:0000016        1
GO:0001733        1
GO:0004799        1
Name: GO_ID, Length: 3951, dtype: int64

In [34]:
for tissue in functional_GO.tissue.unique():
    print(functional_GO[functional_GO.tissue == tissue].GO_ID.value_counts().reset_index())


           index  GO_ID
0     GO:0005515   2406
1     GO:0046872    549
2     GO:0005524    330
3     GO:0042802    291
4     GO:0000978    279
...          ...    ...
2091  GO:0045569      1
2092  GO:0015501      1
2093  GO:0003920      1
2094  GO:0048030      1
2095  GO:0016682      1

[2096 rows x 2 columns]
           index  GO_ID
0     GO:0005515   6211
1     GO:0046872   1055
2     GO:0005524    877
3     GO:0003723    803
4     GO:0042802    775
...          ...    ...
3363  GO:0102751      1
3364  GO:0050104      1
3365  GO:0044016      1
3366  GO:0071820      1
3367  GO:0035226      1

[3368 rows x 2 columns]
           index  GO_ID
0     GO:0005515   4937
1     GO:0046872    916
2     GO:0005524    676
3     GO:0003723    603
4     GO:0042802    602
...          ...    ...
2992  GO:0071209      1
2993  GO:0061656      1
2994  GO:0018549      1
2995  GO:0008832      1
2996  GO:0016682      1

[2997 rows x 2 columns]
           index  GO_ID
0     GO:0005515   2681
1     GO:0003